In [10]:
from langchain.vectorstores import qdrant
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
import getpass
import os
import qdrant_client
from qdrant_client.http import models
from qdrant_client import QdrantClient

### Initilizing the Qdrant Cloud

In [11]:
# Initialize the QdrantClient with the Qdrant server URL and API key

qdrant_client = QdrantClient(
    url=os.getenv('Qdrant_url'),
    api_key=os.getenv("Qdrant_api_key")
)


### Creating a Collection


In [12]:

qdrant_client.create_collection(
    collection_name="Teesney_collection",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
)

True

In [13]:
api_key= os.environ["OpenAI_API_Key"] 
embeddings = OpenAIEmbeddings(api_key=api_key)
vector_store = Qdrant(
    client=qdrant_client, collection_name="Teesney_collection", 
    embeddings=embeddings,
)



### Split the texts into Chunks

In [14]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator = ">>>",
        chunk_size=2100, 
        chunk_overlap= 200,
        length_function=len
        )
    chunks = text_splitter.split_text(text)
    return chunks

In [15]:
with open('story.txt','r', encoding='utf-8') as f:
    raw_text = f.read()

texts=get_chunks(raw_text)
vector_store.add_texts(texts)

['6bf27c4b110d4ca79b6b016a15f897e8',
 '3e7cd653407b4e5581a95ecb98e6e659',
 'f475121c7c5d47728c2765536593f299',
 'a7b2a3fa49a247ed963117abb0c52c4e',
 'cd8b3fb9a3164d6989b5df347c2b4d27',
 '03189bc24ed04733bd479a13ab8fec64',
 '9821e67c63334bb3937f000c26e0600c',
 'e7d3554d7caf4e12ac604473a2fdba1b',
 '247d1b08c81b47a08032d7bf4881d63e',
 'e17b00eea31b4c33991fb6abbdd81859',
 'a43c18a3872b4da1a2ae2c6395da0a1a',
 '6011dec82dab475db6df8be949c789fb',
 '8c3e8fd74f684a36b72f4451dfc9cc59',
 'cb4293a4ea6c4a2193c2eeb5a2ae7658',
 '11b8ca431fa4494881cd76ff66b3833d',
 'cb5aef91378d44c085524df6b0a39a3e',
 'd66b80302b754760b915702c35003399',
 'd26ba49fc0e748b188c68a21dd000eed',
 '17ef6407170a465f96c83eaf16aa9b3c',
 '866068d3ebeb4147a49cdde8cbc219ad',
 '310f5203125e4e07bc7f8f7bec0d40dc',
 'f271232420f3423d90558962e5baabac',
 '0cdc4d89928444f7a39119abacb1c4e6',
 'c25960b11d3b4ad198310e8555307660',
 '9d10cdc26bb848ddad80b3ced6692297',
 '2b8d2f0d0032468cbcf7776c6b51a6b7',
 'a36bc8f0cbbe48f4a8017c91f0eec768',
 

In [17]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
qa = RetrievalQA.from_chain_type( llm=OpenAI(), chain_type = "stuff",
                                 retriever= vector_store.as_retriever()
                                )

In [23]:
querry = "Can you please suggest ma product for a formal wear"
response = qa.invoke(querry)
print(response['result'])

 Yes, the Trucker hat from Stef's exclusive collection would be a great accessory for a formal wear. It features a stylish design and white and pink color combination that adds a touch of freshness to any look. Plus, it is made of 100% Polyester with added sponge for comfort, making it a practical and fashionable choice for any formal event. You can purchase it here: [Trucker hat](https://dev.teesney.com/designers/stef-category/stef-001/kapelo-trucker-el)
